# Credit Risk Resampling Techniques

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [4]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)


#df[["issue_d","next_pymnt_d"]] = pd.to_datetime(df[["issue_d","next_pymnt_d"]]).strftime('%m-%Y')
#df[["issue_d","next_pymnt_d"]] = pd.to_datetime(df[["issue_d","next_pymnt_d"]]).view(int)//10**9

df = df.drop(columns="issue_d")
df = df.drop(columns="next_pymnt_d")

column_names = df.select_dtypes(include="object")
print(column_names)
for column_name in column_names:
    df[column_name] = df[column_name].astype('string')
#df.to_csv("check_data.csv")
column_names = df.select_dtypes(include="object")
print(column_names)
df.head()

      home_ownership verification_status loan_status pymnt_plan  \
0               RENT     Source Verified    low_risk          n   
1           MORTGAGE            Verified    low_risk          n   
2           MORTGAGE            Verified    low_risk          n   
3               RENT            Verified    low_risk          n   
4           MORTGAGE        Not Verified    low_risk          n   
...              ...                 ...         ...        ...   
68812           RENT     Source Verified    low_risk          n   
68813           RENT        Not Verified    low_risk          n   
68814       MORTGAGE     Source Verified    low_risk          n   
68815       MORTGAGE            Verified    low_risk          n   
68816       MORTGAGE            Verified    low_risk          n   

      initial_list_status application_type hardship_flag debt_settlement_flag  
0                       w       Individual             N                    N  
1                       w       Ind

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,low_risk,n,27.24,0.0,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,low_risk,n,20.23,0.0,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,low_risk,n,24.26,0.0,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,low_risk,n,31.44,0.0,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,low_risk,n,18.76,0.0,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
df[["home_ownership", "verification_status"]] = df[["home_ownership", "verification_status"]].astype('category')

df["home_ownership"]
df["verification_status"]
df["home_ownership"].replace(['ANY','MORTGAGE','OWN','RENT'],[0,1,2,3], inplace=True)
df["verification_status"].replace(['Not Verified','Source Verified','Verified'],[0,1,2], inplace=True)

df[["pymnt_plan", "hardship_flag", "debt_settlement_flag","initial_list_status","application_type"]] = df[["pymnt_plan", "hardship_flag", "debt_settlement_flag","initial_list_status","application_type"]].astype('category')
df["pymnt_plan"].replace(['n'],[0], inplace=True)
df["hardship_flag"].replace(['N'],[0], inplace=True)
df["debt_settlement_flag"].replace(['N'],[0], inplace=True)
df["initial_list_status"].replace(['f','w'],[0,1],inplace=True)
df["application_type"].replace(['Individual','Joint App'],[0,1], inplace=True)


# Split the Data into Training and Testing

In [7]:
# Create our target
y = df["loan_status"]

# Create our features
X = df.drop(columns="loan_status")



In [8]:
X.describe()
print(type(X))
print(X.select_dtypes(include=['category']))

<class 'pandas.core.frame.DataFrame'>
      home_ownership verification_status pymnt_plan initial_list_status  \
0                  3                   1          0                   1   
1                  1                   2          0                   1   
2                  1                   2          0                   1   
3                  3                   2          0                   1   
4                  1                   0          0                   1   
...              ...                 ...        ...                 ...   
68812              3                   1          0                   1   
68813              3                   0          0                   1   
68814              1                   1          0                   1   
68815              1                   2          0                   0   
68816              1                   2          0                   1   

      application_type hardship_flag debt_settlement_flag  
0

In [9]:
# Check the balance of our target values
y.head()
print(type(y))
y.value_counts()

<class 'pandas.core.series.Series'>


low_risk     68470
high_risk      347
Name: loan_status, dtype: Int64

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, stratify=y)
X_train.shape

(51612, 83)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [11]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'low_risk': 51352, 'high_risk': 51352})

In [12]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [13]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)



array([[   52,    35],
       [ 5697, 11421]])

In [14]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.6324467892236847

In [15]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.60      0.67      0.02      0.63      0.40        87
   low_risk       1.00      0.67      0.60      0.80      0.63      0.40     17118

avg / total       0.99      0.67      0.60      0.80      0.63      0.40     17205



### SMOTE Oversampling

In [16]:
# Resample the training data with SMOTE
# YOUR CODE HERE

from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
#X_resampled.dropna(axis='columns', how='all', inplace=True)
#X_resampled.dropna(inplace=True)
Counter(y_resampled)

Counter({'low_risk': 51352, 'high_risk': 51352})

In [25]:
column_names = X_resampled.select_dtypes(include='category')
column_names
X_resampled[['home_ownership','verification_status','initial_list_status']] = pd.get_dummies(data=X_resampled, columns=['home_ownership', 'verification_status','initial_list_status'])
X_resampled

ValueError: Columns must be same length as key

In [24]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
#X_resampled.select_dtypes(include='category')
#X_resampled[['home_ownership','verification_status','initial_list_status']]=X_resampled[['home_ownership','verification_status','initial_list_status']].astype('string')
#X_resampled[['home_ownership','verification_status','initial_list_status']]= X_resampled[['home_ownership','verification_status','initial_list_status']].fillna('5')
#X_resampled[['home_ownership','verification_status','initial_list_status']]=X_resampled[['home_ownership','verification_status','initial_list_status']].astype('category')

X_resampled[['home_ownership','verification_status','initial_list_status']] = pd.get_dummies(data=X_resampled, columns=['home_ownership', 'verification_status','initial_list_status'])

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

ValueError: Columns must be same length as key

In [16]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

0.662394124702461

In [17]:
# Display the confusion matrix
# YOUR CODE HERE

array([[   64,    37],
       [ 5283, 11821]])

In [18]:
# Print the imbalanced classification report
# YOUR CODE HERE

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.63      0.69      0.02      0.66      0.44       101
   low_risk       1.00      0.69      0.63      0.82      0.66      0.44     17104

avg / total       0.99      0.69      0.63      0.81      0.66      0.44     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [19]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
# YOUR CODE HERE

Counter({'loan_status': 1})

In [20]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

LogisticRegression(random_state=1)

In [21]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

0.5442166848817717

In [22]:
# Display the confusion matrix
# YOUR CODE HERE

array([[   68,    33],
       [10003,  7101]])

In [23]:
# Print the imbalanced classification report
# YOUR CODE HERE

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.67      0.42      0.01      0.53      0.29       101
   low_risk       1.00      0.42      0.67      0.59      0.53      0.27     17104

avg / total       0.99      0.42      0.67      0.58      0.53      0.27     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [24]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
# YOUR CODE HERE

Counter({'loan_status': 1})

In [25]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

LogisticRegression(random_state=1)

In [26]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

0.6400726134353378

In [27]:
# Display the confusion matrix
# YOUR CODE HERE

array([[  71,   30],
       [7232, 9872]])

In [28]:
# Print the imbalanced classification report
# YOUR CODE HERE

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.70      0.58      0.02      0.64      0.41       101
   low_risk       1.00      0.58      0.70      0.73      0.64      0.40     17104

avg / total       0.99      0.58      0.70      0.73      0.64      0.40     17205

